In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf 
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder
import tensorflow.keras as keras
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

In [2]:
review_df = pd.read_pickle('food_review_places.pickle')

In [3]:
review_df['rating'].value_counts()

5.0    34487
4.0    19041
3.0     8118
2.0     5010
1.0     3539
Name: rating, dtype: int64

In [4]:
review_df.head()

,gPlusPlaceId,rating,reviewerName,reviewText,categories,unixReviewTime,reviewTime,gPlusUserId,name,price,...,hours,phone,closed,gps,lat,long,open_days,Dinner,Lunch,Breakfast
0,112181651134020562716,5.0,jason wagner,stop dinner mahi mahi special fantast servic g...,"[european restaurant, italian restaurant]",2012-10-18 21:56:05,2012-10-18,100000196778399872657,Pasquales Trattoria,None,...,"[[Monday, [['Closed']]], [Tuesday, [['11:30 am...",(905) 338-9090,False,"[43.440331, -79.672851]",43.440331,-79.672851,6.0,1.0,0.0,0.0
1,109391653021436219474,5.0,heather goldsworthy,best coffe collingwood,[cafe],2013-09-12 18:38:15,2013-09-12,100000427343266187570,Espresso Post,None,...,None,(705) 446-1740,False,"[44.499252, -80.216587]",44.499252,-80.216587,NaN,NaN,NaN,NaN
2,109391653021436219474,5.0,nicholas clayton,locat old post offic outlet histor pedigre bes...,[cafe],2013-05-04 01:35:12,2013-05-03,104975196938413074237,Espresso Post,None,...,None,(705) 446-1740,False,"[44.499252, -80.216587]",44.499252,-80.216587,NaN,NaN,NaN,NaN
3,110864889964667973890,5.0,chris johnson,get right everi time love guy,"[delivery restaurant, event planner, business ...",2011-09-15 21:54:03,2011-09-15,100000524810171549476,"Delivered Dish | Portland, OR",$$,...,"[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(503) 239-0100,False,"[45.470342, -122.747085]",45.470342,-122.747085,7.0,1.0,0.0,0.0
4,110864889964667973890,1.0,allison lea,use servic order even show 1 hour 20 minut pro...,"[delivery restaurant, event planner, business ...",2013-11-09 10:28:01,2013-11-08,100537426867747151661,"Delivered Dish | Portland, OR",$$,...,"[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(503) 239-0100,False,"[45.470342, -122.747085]",45.470342,-122.747085,7.0,1.0,0.0,0.0


In [5]:
enc = OneHotEncoder()
reviews = list(review_df['reviewText'])
ratings = np.array(list(review_df['rating']))
# one-hot encode the ratings
ratings = enc.fit_transform(ratings.reshape(-1, 1)).toarray()

In [6]:
split = 0.8
train_len = int(split * len(reviews))
train_reviews = np.array(reviews[:train_len])
train_ratings = np.array(ratings[:train_len])
test_reviews = np.array(reviews[train_len:])
test_ratings = np.array(ratings[train_len:])

In [7]:
max_size = 1000
vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=max_size, output_mode='int', output_sequence_length=20)
vectorize_layer.adapt(train_reviews)

D:\Anaconda\envs\py36gpu\lib\site-packages\tensorflow\python\keras\layers\preprocessing\text_vectorization.py:338: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(preprocessed_data.to_list())


In [8]:
inputs = tf.keras.layers.Input(shape=(1, ), dtype=tf.string, name='text')
outputs = vectorize_layer(inputs)
model = tf.keras.Model(inputs, outputs)
# vocab = np.array(vectorize_layer.get_vocabulary())

In [9]:
encoded_train_reviews = model.predict(train_reviews)
encoded_test_reviews = model.predict(test_reviews)

#### Multi-head attention Block

In [10]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

#### Transformer Block

In [11]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

#### Embedding layer

In [12]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

#### Model

In [18]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(20,))
embedding_layer = TokenAndPositionEmbedding(20, 1001, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(5, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [20]:
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
history = model.fit(encoded_train_reviews, train_ratings, batch_size=32, epochs=20, validation_split=0.2)

Epoch 1/20
1404/1404 [==============================] - 5s 3ms/step - loss: 0.9122 - accuracy: 0.5964 - val_loss: 0.9694 - val_accuracy: 0.5734
Epoch 2/20
1404/1404 [==============================] - 4s 3ms/step - loss: 0.9021 - accuracy: 0.6018 - val_loss: 0.9693 - val_accuracy: 0.5792
Epoch 3/20
1404/1404 [==============================] - 4s 3ms/step - loss: 0.8952 - accuracy: 0.6056 - val_loss: 0.9711 - val_accuracy: 0.5752
Epoch 4/20
1404/1404 [==============================] - 4s 3ms/step - loss: 0.8869 - accuracy: 0.6088 - val_loss: 0.9876 - val_accuracy: 0.5799
Epoch 5/20
1404/1404 [==============================] - 4s 3ms/step - loss: 0.8794 - accuracy: 0.6104 - val_loss: 0.9933 - val_accuracy: 0.5765
Epoch 6/20
1404/1404 [==============================] - 4s 3ms/step - loss: 0.8725 - accuracy: 0.6143 - val_loss: 0.9985 - val_accuracy: 0.5736
Epoch 7/20
1404/1404 [==============================] - 4s 3ms/step - loss: 0.8612 - accuracy: 0.6206 - val_loss: 0.9950 - val_accuracy:

In [22]:
score, acc = model.evaluate(encoded_test_reviews, test_ratings)

439/439 [==============================] - 1s 2ms/step - loss: 1.1208 - accuracy: 0.5587
